In [1]:
from parse_obsidian_vault import ObsidianVault, MarkdownThenRecursiveSplit
from pprint import pprint

In [2]:
# vault_path = "/home/xoph/SlipBoxCopy/Slip Box"
vault_path = "/home/xoph/repos/github/nfroseth/world_graph/test_vault"
splitter = MarkdownThenRecursiveSplit(chunk_size = 512, chunk_overlap = 64)

vault = ObsidianVault(vault_path=vault_path, vault_name="TEST_VAULT")
vault.parse_obsidian_vault(splitter = splitter )
parsed_notes = vault.parsed_notes

Starting the parsing of obsidian vault
0it [00:00, ?it/s]note_properties={'aliases': ['Good Faith Schedule'], 'tags': ['daily_journal', 'finance'], 'obsidian_note_property_id': '/mnt/c/Users/Xoph-Lap/Documents/Slip Box/Slip Box/00 Slip Box/Refinement/Good Faith Schedule.md'}
note_properties={'aliases': ['So', 'Cool'], 'tags': ['daily_journal', 'person', 'person/stranger']}
note_properties={'cuid': 'human202208041012', 'alias': ['👤 Blake Moore', 'Blake Moore', 'Blake', 'BDubs'], 'tags': ['person', 'person/friends', 'person/closefriends', 'person/friends/close']}
3it [00:00, 342.30it/s]
Finished parsing notes


In [5]:
for idx, doc in enumerate(parsed_notes["Good Faith Schedule"].properties["chunks"]):
    # for line in doc.page_content.split("\n"):
    print(f"=== Chunk idx:{idx}")
    print(doc)

=== Chunk idx:0
page_content='202201150130  \nStatus: #idea  \nTags  \nFleeting Notes on:'
=== Chunk idx:1
page_content='# Good Faith Schedule\nElaborating on [[Good Faith Scheduling with Time blocks]]' metadata={'Header 1': 'Good Faith Schedule'}
=== Chunk idx:2
page_content='#### Questions to guide\nWhat is my one thing?\nWhat are the principles I ought to use when creating a schedule/routine?\ntherhe' metadata={'Header 1': 'Good Faith Schedule', 'Header 4': 'Questions to guide'}
=== Chunk idx:3
page_content='#### Principles/Constraints  \nFor good health:\nI will attempt to[[Sleep]] 9 hours every night.  \nFor good lifting:  \nAn excerpt to build on from [[Good Faith Scheduling with Time blocks#^70e04e]]  \nKey: {Time Allotment } [[Links to Elaboration]] (Explanation of implementation)  \n**Mornings Routine Start** {2h 20m}\n- Wake up at 8:15 am {5 m}\n- Make bed\n- Piss\n- Take morning [[Supplements]]\n- Brush teeth\n- [[Weight myself]]\n- Get clothing for the day (Picked out the S

In [4]:
print(parsed_notes["Good Faith Schedule"].out_relationships["Food"][0])

inline{'title': 'Food', 'header': '', 'chunk_index': 4, 'parsed_context': '', 'link_display_text': 'Food'}
